In [1]:
import MeCab
import subprocess
import gensim
from gensim.models import word2vec
import numpy as np
import re
from seq2seq.models import AttentionSeq2Seq

ModuleNotFoundError: No module named 'recurrentshop.backend.backend'

In [ ]:
import keras
keras.__version__

In [ ]:
from keras.backend import backend

In [ ]:
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0} -Owakati".format(path))

model_wv = word2vec.Word2Vec.load("./wiki.model").wv

In [ ]:
from keras.backend import tensorflow_backend

In [ ]:
!pip install keras

In [ ]:
def line2vec(text, length=20):
    url = re.compile(r"http(s)?://([\w\-]+\.)+[\w\-]+(\/[\w\- ./?%&=]*)?")
    # 記号類
    text = re.sub("[！!]+", "!", text)
    text = re.sub("[？?]+", "?", text)
    #text = zen2han(text, kana=False)
    #text = han2zen(text, ascii_=False, digit=False, kakko=False)
    text = re.sub("\d+", "0", text)
    text = text.lower()
    text = re.sub("w+", "w", text)
    text = url.sub("", text)

    
    parsed = m.parse(text).strip().split(" ")

    vector = [model_wv[p] for p in parsed if p in model_wv]
    if length:
        if length < len(vector):
            vector = vector[len(vector) - length::]
        else:
            for _ in range(length - len(vector)):
                vector.append(model_wv["。"])
    return vector


def vec2line(vector):
    text = ""
    for v in vector:
        text += model_wv.most_similar([v], [], 1)[0][0]
    return text

In [ ]:
vec=line2vec("隣の客はよく柿食う客だ")

In [ ]:
len(vec)

In [ ]:
vec2line(vec)

In [ ]:
with open("Tweet_data.txt","r") as fp:
    texts=fp.readlines()

In [ ]:
vectors=np.asarray([line2vec(text.strip(), length=140) for text in texts])

In [ ]:
vectors.shape

In [ ]:
g = AttentionSeq2Seq(input_dim=200,input_length=1, output_length=140, output_dim=200)
g.compile(loss='mse', optimizer='Adam')

In [ ]:
epochs=150
for epoch in tqdm(range(epochs)):
    model.fit(np.random(vectors.shape[0],1,200),vectors)
    